# Random Forest from Scratch

In this notebook, I will be entirely covering topics relating to the Random Forest Algorithm. This will start with a simple decision tree implementation, and go over various topics. The dataset that will be used is a NASA star type classfication dataset, where the goal is to have our tree predict the correct star classification given its tree of rules. 

## Importing and Prepping Data

Temperature -- K <br>
L -- L/Lo<br>
R -- R/Ro<br>
AM -- Mv<br>
Color -- General Color of Spectrum<br>
Spectral_Class -- O,B,A,F,G,K,M / SMASS - https://en.wikipedia.org/wiki/Asteroid_spectral_types<br>
Type -- Red Dwarf, Brown Dwarf, White Dwarf, Main Sequence , Super Giants, Hyper Giants<br>

In [60]:
import numpy as np
import pandas as pd

df = pd.read_csv("C:\\Users\\rogerree\\OneDrive - Merck Sharp & Dohme LLC\\Documents\\Personal Projects\\Data\\Stars.csv")
df.shape

(240, 7)

In [61]:
df.dtypes

Temperature         int64
L                 float64
R                 float64
A_M               float64
Color              object
Spectral_Class     object
Type                int64
dtype: object

In [62]:
df.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


In [63]:
# a decision tree/ random forest can work with both categorical variables, such as color, as well
# numerical attributes, but we will need to alter our categorical attributes before passing data
# into a model. 

# for color, one of our categorical variables, we will use one hot encoding, as there is no
# inherent or ordinal relationship between the colors (one color is not greater than another). We
# will just do the same for spectral class as well.

df = pd.get_dummies(df, columns=['Color', 'Spectral_Class'], dtype=int)

# Move the "Type" column to the end
type_column = df.pop("Type")
df["Type"] = type_column

df.head()

,Temperature,L,R,A_M,Color_Blue,Color_Blue White,Color_Blue white,Color_Blue-White,Color_Blue-white,Color_Orange,...,Color_yellow-white,Color_yellowish,Spectral_Class_A,Spectral_Class_B,Spectral_Class_F,Spectral_Class_G,Spectral_Class_K,Spectral_Class_M,Spectral_Class_O,Type
0,3068,0.002400,0.1700,16.12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3042,0.000500,0.1542,16.60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2600,0.000300,0.1020,18.70,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2800,0.000200,0.1600,16.65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1939,0.000138,0.1030,20.06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [64]:
# scaling is not necessary in decision tree, because we will locate the splits that give the best
# reduction in entropy/ most information gain

## Decision Trees

Knowing and understanding the way a decision tree works, our decision tree from scratch will consist of the following parts:

- Node class: represents a node of the decision tree. Each node will have attributes like a split dimension (what attribute this node was split with), split point (what value of that split dimension was setteled on), and a label (which would give the class prediction if the node is a leaf), etc.

- Entropy Computation: compute the entropy of a set of labels

- Split Information: compute split information for a specific split point

- Fitting the Tree: builds the desision tree model recursively 

- Classify: uses built model to predict labels of data

### Decision Tree from Scratch

In [72]:
from typing import List, Tuple
import math
from collections import Counter


class Node:
    def __init__(self, split_dim=None, split_point=None, label=None):
        self.split_dim = split_dim
        self.split_point = split_point
        self.label = label
        self.left = None
        self.right = None
        
    def is_leaf(self):
        return self.label is not None
    
#_______________________________________________________________________________________________

class Solution:
    def __init__(self):
        self.root = None

    # computes the information (entropy) 
    def compute_info(self, labels: pd.Series) -> float:
        total_samples = len(labels)
        label_counts = labels.value_counts()
        info = 0.0

        # this is the key line: information is the negative sum of the probablity of each class times 
        # the log of that same probablity value for each class
        for count in label_counts.values:
            probability = count / total_samples
            info -= probability * math.log2(probability)

        return info

    def split_info(self, data: pd.DataFrame, label: pd.Series, split_dim: int, split_point: float) -> float:
        # left and right just represent the two subsets that are created when we split the data. We can 
        # capture the data that is in each subset, as well as the labels
        data_left = data[data.iloc[:, split_dim] <= split_point]
        data_right = data[data.iloc[:, split_dim] > split_point]
        label_left = label[data.iloc[:, split_dim] <= split_point]
        label_right = label[data.iloc[:, split_dim] > split_point]
        # within this code, we are checking each value of the input value against the split point to 
        # decide if it goes to the left or right subset

        total_samples = len(data)
        total_left = len(data_left)
        total_right = len(data_right)

        p_left = total_left / total_samples
        p_right = total_right / total_samples

        info_left = self.compute_info(label_left)
        info_right = self.compute_info(label_right)
        # we then weigh the information we gather from each side's split, and return the sum of those 
        # as info_a
        info_a = (p_left * info_left) + (p_right * info_right)

        return info_a

#_______________________________________________________________________________________________
    def _split_data(self, data: pd.DataFrame, labels: pd.Series, split_dim: int, split_point: float) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
        data_left = data[data.iloc[:, split_dim] <= split_point]
        data_right = data[data.iloc[:, split_dim] > split_point]
        labels_left = labels[data.iloc[:, split_dim] <= split_point]
        labels_right = labels[data.iloc[:, split_dim] > split_point]

        return data_left, labels_left, data_right, labels_right

    def _calculate_info_gain(self, data: pd.DataFrame, labels: pd.Series, split_dim: int, split_point: float) -> float:
        total_samples = len(data)
        total_left, total_right = 0, 0
        count_left, count_right = labels[data.iloc[:, split_dim] <= split_point].value_counts(), labels[data.iloc[:, split_dim] > split_point].value_counts()

        info = self.compute_info(labels)
        info_left = 0.0
        info_right = 0.0

        for count in count_left.values:
            probability = count / total_samples
            info_left -= probability * math.log2(probability)

        for count in count_right.values:
            probability = count / total_samples
            info_right -= probability * math.log2(probability)

        info_gain = info - ((count_left.sum() / total_samples) * info_left + (count_right.sum() / total_samples) * info_right)
        return info_gain
    
    def fit(self, train_data: pd.DataFrame, train_label: pd.Series) -> None:
        self.root = self._recursive_build(train_data, train_label)

    def _recursive_build(self, data: pd.DataFrame, labels: pd.Series, depth: int = 1) -> Node:
        label_counts = labels.value_counts()
        majority_label = label_counts.idxmax()

        if len(label_counts) == 1 or depth > 2:
            return Node(label=majority_label, split_dim=-1, split_point=-1.0)

        num_features = data.shape[1]
        best_info_gain = float('-inf')
        best_split_dim = -1
        best_split_point = -1

        for split_dim in range(num_features):
            split_points = self._calculate_split_points(data, split_dim)

            for split_point in split_points:
                info_gain = self._calculate_info_gain(data, labels, split_dim, split_point)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split_dim = split_dim
                    best_split_point = split_point

        node = Node(split_dim=best_split_dim, split_point=best_split_point, label=majority_label)

        data_left, labels_left, data_right, labels_right = self._split_data(data, labels, best_split_dim, best_split_point)
        if not data_left.empty and not data_right.empty:
            node.left = self._recursive_build(data_left, labels_left, depth=depth + 1)
            node.right = self._recursive_build(data_right, labels_right, depth=depth + 1)

        return node

    def _calculate_split_points(self, data: pd.DataFrame, split_dim: int) -> List[float]:
        attribute_values = sorted(data.iloc[:, split_dim].unique())
        split_points = [(attribute_values[i] + attribute_values[i + 1]) / 2 for i in range(len(attribute_values) - 1)]
        return split_points
    
#_______________________________________________________________________________________________

    def classify(self, train_data: List[List[float]], train_label: List[int], test_data: List[List[float]]) -> List[int]:
        self.fit(train_data, train_label)
        predictions = []

        for data_point in test_data:
            predicted_label = self._traverse_tree(data_point, self.root)
            predictions.append(predicted_label)

        return predictions

    def _traverse_tree(self, data_point: List[float], node: Node) -> int:
        if node.left is None and node.right is None:
            return node.label
        if data_point[node.split_dim] <= node.split_point:
            return self._traverse_tree(data_point, node.left)
        else:
            return self._traverse_tree(data_point, node.right)

In [66]:
# Testing for Entropy and Split Info

def parse_input_dataframe(df):
    split_dim = 3  # Set the split_dim (dimension to split on) to 0 as an example
    split_point = 13  # Set the split_point to 0.5 as an example

    data = df.iloc[:15]  # Take the first 20 records and convert them to a list of lists
    labels = df.iloc[:15]['Type']  # Extract the 'Type' column as the labels

    return data, labels, split_dim, split_point

data, labels, split_dim, split_point = parse_input_dataframe(df)

solution = Solution()

# Call the split_info method with the parsed data
result = solution.split_info(data, labels, split_dim, split_point)
print(result)

0.32229779040910983


In [71]:
# Testing for the tree creation and fit

dataset = df.iloc[:20, :-1]
labels = df.iloc[:20, -1]

# Create an instance of the Solution class
solution = Solution()

# Build the decision tree using the fit function
solution.fit(dataset, labels)

def preorder_traversal(node):
    if node is None:
        return ""
    result = "{" + f"split_dim: {node.split_dim}, split_point: {node.split_point}, label: {node.label}" + "}"
    if node.left or node.right:
        result += "{" + preorder_traversal(node.left) + "}"
        result += "{" + preorder_traversal(node.right) + "}"
    return result

def inorder_traversal(node):
    if node is None:
        return ""
    result = ""
    if node.left or node.right:
        result += "{" + inorder_traversal(node.left) + "}"
    result += "{" + f"split_dim: {node.split_dim}, split_point: {node.split_point}, label: {node.label}" + "}"
    if node.left or node.right:
        result += "{" + inorder_traversal(node.right) + "}"
    return result

# Perform preorder traversal on the decision tree
preorder_result = preorder_traversal(solution.root)
print(preorder_result)

print()

# Perform inorder traversal on the decision tree
inorder_result = inorder_traversal(solution.root)
print(inorder_result)

{split_dim: 3, split_point: 15.42, label: 0}{{split_dim: -1, split_point: -1.0, label: 1}}{{split_dim: -1, split_point: -1.0, label: 0}}

{{split_dim: -1, split_point: -1.0, label: 1}}{split_dim: 3, split_point: 15.42, label: 0}{{split_dim: -1, split_point: -1.0, label: 0}}


### Information Gain and Entropy

### Building the Tree and Structure

### Pruning and Overfitting

## Random Forests

### Random Forest as an Ensemble of Decision Trees

### Random Subspace Method for Feature Selection

### Bootstrapping 

### Hyperparameter Tuning

## Evaluation of Models and Cross-validation

## Variations and Improvements

### Feature Importance Estimation with Random Forest

### Handling Missing Data and Outliers

### Extending Random Forest to Handle Imbalanced Datasets

### Multiclass Classification and Regression Problems

## Coursework for Random Forest

In [193]:
# Classification Test

def parse_classification_data(file_path):
    with open(file_path, 'r') as file:
        train_data = []
        train_label = []
        test_data = []
        
        for line in file:
            line = line.strip()
            parts = line.split()
            label = int(parts[0])
            attributes = [float(attr.split(':')[1]) for attr in parts[1:]]
            
            if label != -1:
                train_data.append(attributes)
                train_label.append(label)
            else:
                test_data.append(attributes)
    
    return train_data, train_label, test_data

input_file = r"C:\Users\rogerree\Downloads\01mSTLanT0igEUUS8kDmDQ_2834ea673e8949dea706a3da771f23f1_PA-DT-Handout\dt_handout\sample_test_cases\classification\input01.txt"
train_data, train_label, test_data = parse_classification_data(input_file)
result = solution.classify(train_data, train_label, test_data)
solution = Solution()


# Call the classify method with the extracted data
result = solution.classify(train_data, train_label, test_data)
print(result)

[2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2, 3, 3, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3]
